# Food Recognition Model Architecture

This notebook defines the model architecture for food recognition using transfer learning with EfficientNetV2.
  

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2L
from tensorflow.keras import layers, Model
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

In [2]:
# Define constants
IMG_SIZE = 224
NUM_CLASSES = 101
BATCH_SIZE = 32
EPOCHS = 50

# Define paths
PROCESSED_DATA_PATH = Path('../processed_data')
MODEL_SAVE_PATH = Path('../models')
MODEL_SAVE_PATH.mkdir(exist_ok=True)

In [3]:
# Create data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    PROCESSED_DATA_PATH / 'train',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    PROCESSED_DATA_PATH / 'val',
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
  

Found 60600 images belonging to 101 classes.
Found 20200 images belonging to 101 classes.


In [4]:
# Create the base model
def create_model():
    # Load the pre-trained model
    base_model = EfficientNetV2L(
        weights='imagenet',
        include_top=False,
        input_shape=(IMG_SIZE, IMG_SIZE, 3)
    )
    
    # Freeze the base model layers
    base_model.trainable = False
    
    # Create the model
    model = tf.keras.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(0.2),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.2),
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    return model

# Create and compile the model
model = create_model()
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Display model summary
model.summary()

2025-04-01 18:05:02.880371: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-04-01 18:05:02.880409: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-04-01 18:05:02.880413: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-04-01 18:05:02.880586: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-01 18:05:02.880597: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 16s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetv2-l (Functional)   │ (None, 7, 7, 1280)     │   117,746,848 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       655,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 101)            │        51,813 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,454,533 (451.87 MB)

 Trainable params: 707,685 (2.70 MB)

 Non-trainable params: 117,746,848 (449.17 MB)

In [5]:
# Define callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=3,
        min_lr=0.00001
    ),
    tf.keras.callbacks.ModelCheckpoint(
        filepath=MODEL_SAVE_PATH / 'best_model.h5',
        monitor='val_accuracy',
        save_best_only=True
    )
]

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

/Users/vlzqz/Desktop/escuela/spring 2025/440 social networks/OpenEndedResearch/.venv/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50


2025-04-01 18:05:29.361328: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1140/1894 ━━━━━━━━━━━━━━━━━━━━ 16:41 1s/step - accuracy: 0.0105 - loss: 4.7254

KeyboardInterrupt: 

In [ ]:
# Plot training history
def plot_training_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy')
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
    ax1.set_title('Model Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss')
    ax2.plot(history.history['val_loss'], label='Validation Loss')
    ax2.set_title('Model Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

plot_training_history(history)

## Model Architecture Summary

1. Base Model:
   - EfficientNetV2L (Large) pre-trained on ImageNet
   - Input size: 224x224x3
   - Base model layers are frozen for transfer learning

2. Added Layers:
   - Global Average Pooling
   - Dropout (0.2)
   - Dense layer (512 units, ReLU)
   - Dropout (0.2)
   - Output layer (101 units, softmax)

3. Training Configuration:
   - Optimizer: Adam (lr=0.001)
   - Loss: Categorical Cross-entropy
   - Metrics: Accuracy
   - Batch size: 32
   - Max epochs: 50

4. Callbacks:
   - Early stopping (patience=5)
   - Learning rate reduction (factor=0.2, patience=3)
   - Model checkpointing (saves best model)

Next Steps:
1. Fine-tune the model by unfreezing some base model layers
2. Implement model evaluation on test set
3. Create inference pipeline for new images